<a href="https://colab.research.google.com/github/descartesmbogning/Deciphering_Bacteriophage_Revolution/blob/main/1_patent_analysis_trend2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re
from functools import reduce
import seaborn as sns
import missingno as msno
%matplotlib inline

import matplotlib.pyplot as plt
from scipy.stats import norm


In [2]:
# Set the figure size - handy for larger output
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = [10, 6]
# Set up with a higher resolution screen (useful on Mac)
%config InlineBackend.figure_format = 'retina'

In [3]:
from scipy import stats #the only import you actually need for this
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}

In [4]:
# function that tests distribution then picks test accordingly
def test_sig(x, y):
    if stats.shapiro(x).pvalue >= 0.05 and stats.shapiro(y).pvalue >= 0.05: #if they are normally distriuted
        print("t-test")
        return stats.ttest_ind(x,y)
    else:
        print("mann-whitney")
        return stats.mannwhitneyu(x, y)

## import data

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
#Import data
data = pd.read_pickle(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/topic_probability_patent.pkl').reset_index()
# Convert 'Publication Year' to integers
data['Publication Year'] = data['Publication Year'].astype(int)

data.head()

,Lens ID,Publication Year,#,Jurisdiction,Kind,Display Key,Publication Date,Application Number,Application Date,Priority Numbers,...,IPCR Classifications,US Classifications,NPL Citation Count,NPL Resolved Citation Count,NPL Resolved Lens ID(s),NPL Resolved External ID(s),NPL Citations,Legal Status,Cleaned_Text,Topic N°
0,088-886-194-310-021,2006,6343,FR,A1,FR 2878165 A1,26/05/2006,FR 0412364 A,22/11/2004,FR 0412364 A,...,A61K38/17;;A61K39/395;;A61K48/00;;A61P9/00;;A6...,NaN,1,1,085-906-838-714-934,11038171;;pmc2192657;;10.1083/jcb.151.2.221,"KOCH M ET AL: ""A NOVEL MEMBER OF THE NETRIN FA...",DISCONTINUED,"new (mutant) netrin-4, netrin-1, netrin-3, ant...",4
1,147-331-231-570-922,2007,2680,WO,A1,WO 2007/083793 A1,26/07/2007,JP 2007050890 W,22/01/2007,JP 2006014314 A,...,G01N33/53;;C07D229/02;;G01N33/15;;G01N33/543;;...,NaN,2,1,016-994-266-998-256,10.2174/1568026023394182;;11944820,"HATANAKA Y. ET AL.: ""Photoaffinity Labeling in...",PENDING,panning method utilizing photoreactive group k...,2
2,148-164-161-664-368,2007,111,WO,A1,WO 2007/114139 A1,11/10/2007,JP 2007056579 W,28/03/2007,JP 2006105143 A,...,C12N15/09;;C07K14/005;;C12N1/21;;C12N7/00;;C12...,NaN,24,22,050-417-296-901-112;;051-984-368-264-261;;058-...,16277371;;10.1021/cr000261r;;10.1006/jmbi.1999...,"KEHOE J.W. ET AL.: ""Filamentous phage display ...",PENDING,phage display novel filamentous bacteriophage[...,2
3,161-684-752-020-329,2004,3085,JP,A,JP 2004097032 A,02/04/2004,JP 2002260418 A,05/09/2002,JP 2002260418 A,...,C12N15/09;;C12Q1/04;;C12Q1/34,NaN,0,0,NaN,NaN,NaN,EXPIRED,method monitoring bacterium utilizing chitinou...,0
4,089-102-410-621-069,2004,3892,JP,A,JP 2004099465 A,02/04/2004,JP 2002260419 A,05/09/2002,JP 2002260419 A,...,C12N15/09;;A01N25/28;;A01N63/00;;C12N1/20;;C12...,NaN,0,0,NaN,NaN,NaN,EXPIRED,method controlling insect pest alginate microb...,5


In [7]:
# Check the number of patents
num_patents = data.shape[0]
print(f"Number of Patents: {num_patents}")


Number of Patents: 5108


# Patent trends

In [8]:
data_trend = data['Publication Year'].value_counts().reset_index().sort_values('index')
# Calculate the inverse cumulative proportion
#data_trend['Inverse Cumulative Proportion'] = data_trend['Publication Year'].iloc[::-1].cumsum() / data_trend['Publication Year'].sum()

data_trend

,index,Publication Year
20,2003,112
15,2004,156
18,2005,115
16,2006,126
17,2007,119
19,2008,114
12,2009,172
14,2010,156
9,2011,192
11,2012,176


In [9]:
import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(data_trend, *['index', 'Publication Year', None], **{})
chart

# Applicants

In [10]:
#Import applicants_data
applicants_data = pd.read_pickle(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/topic_probability_patent.pkl').reset_index()[
    ['Lens ID','Publication Year','Applicants','Cited by Patent Count']]
# Convert 'Publication Year' to integers
applicants_data['Publication Year'] = applicants_data['Publication Year'].astype(int)

applicants_data = applicants_data.set_index(['Lens ID','Publication Year','Cited by Patent Count'])
applicants_data

,,,Applicants
Lens ID,Publication Year,Cited by Patent Count,
088-886-194-310-021,2006,7,CENTRE NAT RECH SCIENT;;I V S INST DES VAISSEA...
147-331-231-570-922,2007,2,UNIV TOYAMA NAT UNIV CORP;;HATANAKA YASUMARU;;...
148-164-161-664-368,2007,0,UCHIYAMA FUMIAKI
161-684-752-020-329,2004,1,UNIV KOBE
089-102-410-621-069,2004,0,UNIV KOBE
...,...,...,...
179-151-089-030-564,2019,0,UNIV MASARYKOVA
014-594-233-943-253,2020,1,MB PHARMA S R O
115-179-292-453-581,2006,1,NEW HORIZONS DIAGNOSTICS CORPORATION;;UNIV ROC...


In [11]:
import pandas as pd

# Your DataFrame is named applicants_data
applicants_list = applicants_data['Applicants'].str.split(';;').explode().str.strip()

# Count the unique applicants names
unique_applicants_count = applicants_list.nunique()

print('The number of unique applicants is:', unique_applicants_count)

#
applicants_list2 = applicants_list.to_frame().reset_index()

# and sum citations within each group
grouped = applicants_list2.groupby(['Applicants']).agg({'Lens ID': 'count', 'Cited by Patent Count': 'sum'}).reset_index()

grouped['%'] = grouped['Lens ID']/applicants_data['Applicants'].count()*100
grouped['Average Citation Count'] = grouped['Cited by Patent Count']/grouped['Lens ID']
# Rename columns
grouped.columns = ['Applicants', 'count','citation', '%',  'Average Citation Count']

# Save data
grouped.to_pickle(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/applicants_count_citation.pkl')
grouped.to_csv(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/applicants_count_citation.csv')

grouped#.head(50)

The number of unique applicants is: 3983


,Applicants,count,citation,%,Average Citation Count
0,(주)일신이디아이,1,0,0.019798,0.0
1,13 8 INC,1,0,0.019798,0.0
2,2ND AF HOSP GUANGZHOU MED UNIV,1,2,0.019798,2.0
3,3M INNOVATIVE PROPERTIES CO,1,0,0.019798,0.0
4,48HOUR DISCOVERY INC,1,0,0.019798,0.0
...,...,...,...,...,...
3978,ZYMO RES CORP,1,4,0.019798,4.0
3979,“UKRAINIAN MEDICAL STOMATOLOGICAL ACADEMY” HIG...,1,0,0.019798,0.0
3980,서울대학교산학협력단,2,0,0.039596,0.0
3981,이상훈,1,0,0.019798,0.0


In [12]:
# Reset the index to make 'Publication Year' a regular column
applicants_list2 = applicants_list.to_frame().reset_index()
# Create a new column, 'Year Group', to indicate which 5-year group each row belongs to
applicants_list2['Year Group'] = pd.cut(applicants_list2['Publication Year'], bins=range(2003, 2035, 5), right=False)


# Group by 'Year Group' and '......', count number of publications
# and sum citations within each group
grouped = applicants_list2.groupby(['Year Group','Applicants']).agg({'Lens ID': 'count', 'Cited by Patent Count': 'sum'}).reset_index()
grouped['Average Citation Count'] = grouped['Cited by Patent Count']/grouped['Lens ID']
# Create a boolean mask to identify rows with 0 in the 'Lens ID' column
mask = (grouped['Lens ID'] == 0)

# Use the mask to drop rows with 0 in the 'Lens ID' column
grouped_filtered = grouped[~mask]

# Rename columns
grouped_filtered.columns = ['Year Group','Applicants', 'Publication Count', 'Total Citations',  'Average Citation Count']

grouped_filtered


,Year Group,Applicants,Publication Count,Total Citations,Average Citation Count
8,"[2003, 2008)",AARON DIAMOND AIDS RES CT,1,0,0.0
22,"[2003, 2008)",ACTIVE BIOTECH AB,1,2,2.0
24,"[2003, 2008)",ADANG MICHAEL J.,1,3,3.0
29,"[2003, 2008)",ADHYA SANKAR,1,35,35.0
30,"[2003, 2008)",ADHYA SANKAR L.,1,12,12.0
...,...,...,...,...,...
19759,"[2023, 2028)",YANCHENG INST TECH,1,0,0.0
19782,"[2023, 2028)",YEDA RES & DEV,1,0,0.0
19804,"[2023, 2028)",YOUYIBANG BIOTECHNOLOGY SHANGHAI CO LTD,1,0,0.0
19819,"[2023, 2028)",ZAMAT ALI,1,0,0.0


In [13]:
# Pivot the DataFrame
pivot_df = grouped_filtered.pivot(index='Applicants', columns='Year Group', values=['Publication Count', 'Total Citations',  'Average Citation Count'])

# Fill NaN values with 0
#pivot_df.fillna(0, inplace=True)
# Save data
pivot_df.to_pickle(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/applicants_list_count_citation_group_year.pkl')
pivot_df.to_csv(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/applicants_list_count_citation_group_year.csv')

# Print the pivoted DataFrame
pivot_df


Publication Count  \
Year Group                                              [2003, 2008)   
Applicants                                                             
(주)일신이디아이                                                        NaN   
13 8 INC                                                         NaN   
2ND AF HOSP GUANGZHOU MED UNIV                                   NaN   
3M INNOVATIVE PROPERTIES CO                                      NaN   
48HOUR DISCOVERY INC                                             NaN   
...                                                              ...   
ZYMO RES CORP                                                    NaN   
“UKRAINIAN MEDICAL STOMATOLOGICAL ACADEMY” HIGH...               NaN   
서울대학교산학협력단                                                       NaN   
이상훈                                                              NaN   
주식회사 비엠씨                                                         NaN   

                                                                              \
Year Group                                         [2008, 2013) [2013, 2018)   
Applicants                                                                     
(주)일신이디아이                                                   NaN          NaN   
13 8 INC                                                    NaN          NaN   
2ND AF HOSP GUANGZHOU MED UNIV                              1.0          NaN   
3M INNOVATIVE PROPERTIES CO                                 NaN          NaN   
48HOUR DISCOVERY INC                                        NaN          NaN   
...                                                         ...          ...   
ZYMO RES CORP                                               1.0          NaN   
“UKRAINIAN MEDICAL STOMATOLOGICAL ACADEMY” HIGH...          NaN          1.0   
서울대학교산학협력단                                                  NaN          1.0   
이상훈                                                         1.0          NaN   
주식회사 비엠씨                                                    NaN          1.0   

                                                                              \
Year Group                                         [2018, 2023) [2023, 2028)   
Applicants                                                                     
(주)일신이디아이                                                   1.0          NaN   
13 8 INC                                                    1.0          NaN   
2ND AF HOSP GUANGZHOU MED UNIV                              NaN          NaN   
3M INNOVATIVE PROPERTIES CO                                 1.0          NaN   
48HOUR DISCOVERY INC                                        1.0          NaN   
...                                                         ...          ...   
ZYMO RES CORP                                               NaN          NaN   
“UKRAINIAN MEDICAL STOMATOLOGICAL ACADEMY” HIGH...          NaN          NaN   
서울대학교산학협력단                                                  1.0          NaN   
이상훈                                                         NaN          NaN   
주식회사 비엠씨                                                    NaN          NaN   

                                                   Total Citations  \
Year Group                                            [2003, 2008)   
Applicants                                                           
(주)일신이디아이                                                      NaN   
13 8 INC                                                       NaN   
2ND AF HOSP GUANGZHOU MED UNIV                                 NaN   
3M INNOVATIVE PROPERTIES CO                                    NaN   
48HOUR DISCOVERY INC                                           NaN   
...                                                            ...   
ZYMO RES CORP                                                  NaN   
“UKRAINIAN MEDICAL STOMATOLOGICAL ACADEMY” HIGH...             NaN   
서울대학교산학협력단                    

# Inventors

In [14]:
#Import inventors_data
inventors_data = pd.read_pickle(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/topic_probability_patent.pkl').reset_index()[
    ['Lens ID','Publication Year','Inventors','Cited by Patent Count']]
# Convert 'Publication Year' to integers
inventors_data['Publication Year'] = inventors_data['Publication Year'].astype(int)

inventors_data = inventors_data.set_index(['Lens ID','Publication Year','Cited by Patent Count'])

inventors_data

,,,Inventors
Lens ID,Publication Year,Cited by Patent Count,
088-886-194-310-021,2006,7,PLOUET JEAN;;ALEMANY MONICA
147-331-231-570-922,2007,2,HATANAKA YASUMARU;;SADAKANE YUTAKA
148-164-161-664-368,2007,0,UCHIYAMA FUMIAKI
161-684-752-020-329,2004,1,MAYAMA SHIGEYUKI;;TOSA YUKIO;;OTSU YASUNARI;;T...
089-102-410-621-069,2004,0,MAYAMA SHIGEYUKI;;TOSA YUKIO;;OTSU YASUNARI;;T...
...,...,...,...
179-151-089-030-564,2019,0,PANTŮČEK ROMAN;;ŠTVERÁKOVÁ DANA;;ŠEDO ONDREJ;;...
014-594-233-943-253,2020,1,MOŠA MAREK;;BENEŠÍK MARTIN
115-179-292-453-581,2006,1,FISCHETTI VINCENT;;LOOMIS LAWRENCE


In [15]:
import pandas as pd

# Your DataFrame is named inventors_data
inventors_list = inventors_data['Inventors'].str.split(';;').explode().str.strip()

# Count the unique inventors names
unique_inventors_count = inventors_list.nunique()

print('The number of unique inventors is:', unique_inventors_count)

#
inventors_list2 = inventors_list.to_frame().reset_index()

# and sum citations within each group
grouped = inventors_list2.groupby(['Inventors']).agg({'Lens ID': 'count', 'Cited by Patent Count': 'sum'}).reset_index()

grouped['%'] = grouped['Lens ID']/inventors_data['Inventors'].count()*100
grouped['Average Citation Count'] = grouped['Cited by Patent Count']/grouped['Lens ID']
# Rename columns
grouped.columns = ['Inventors', 'count','citation', '%',  'Average Citation Count']

# Save data
grouped.to_pickle(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/inventors_count_citation.pkl')
grouped.to_csv(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/inventors_count_citation.csv')

grouped#.head(50)

The number of unique inventors is: 12365


,Inventors,count,citation,%,Average Citation Count
0,A MINA,1,4,0.019712,4.000000
1,A P NETHRAVATHI,1,0,0.019712,0.000000
2,AAGE LOESET GEIR,1,2,0.019712,2.000000
3,ABAEV IGOR V,3,1,0.059137,0.333333
4,ABAEV IGOR VALENTINOVICH,1,1,0.019712,1.000000
...,...,...,...,...,...
12360,유상열,1,0,0.019712,0.000000
12361,이상훈,1,0,0.019712,0.000000
12362,이유나,1,0,0.019712,0.000000
12363,조도형,1,0,0.019712,0.000000


In [16]:
# Reset the index to make 'Publication Year' a regular column
inventors_list2 = inventors_list.to_frame().reset_index()
# Create a new column, 'Year Group', to indicate which 5-year group each row belongs to
inventors_list2['Year Group'] = pd.cut(inventors_list2['Publication Year'], bins=range(2003, 2035, 5), right=False)


# Group by 'Year Group' and '......', count number of publications
# and sum citations within each group
grouped = inventors_list2.groupby(['Year Group','Inventors']).agg({'Lens ID': 'count', 'Cited by Patent Count': 'sum'}).reset_index()
grouped['Average Citation Count'] = grouped['Cited by Patent Count']/grouped['Lens ID']
# Create a boolean mask to identify rows with 0 in the 'Lens ID' column
mask = (grouped['Lens ID'] == 0)

# Use the mask to drop rows with 0 in the 'Lens ID' column
grouped_filtered = grouped[~mask]

# Rename columns
grouped_filtered.columns = ['Year Group','Inventors', 'Publication Count', 'Total Citations',  'Average Citation Count']

grouped_filtered


,Year Group,Inventors,Publication Count,Total Citations,Average Citation Count
12,"[2003, 2008)",ACOSTA ANABEL ALVAREZ,1,0,0.000000
17,"[2003, 2008)",ADANG MICHAEL J,1,3,3.000000
21,"[2003, 2008)",ADHYA SANKAR,4,43,10.750000
22,"[2003, 2008)",ADHYA SANKAR L,1,12,12.000000
24,"[2003, 2008)",AEVARSSON ARNTHOR,3,38,12.666667
...,...,...,...,...,...
61679,"[2023, 2028)",ZHU SHUJIAO,2,0,0.000000
61712,"[2023, 2028)",ZHU ZHIXIAN,1,0,0.000000
61714,"[2023, 2028)",ZHU ZISHAN,1,0,0.000000
61756,"[2023, 2028)",ZOU LING,1,0,0.000000


In [17]:
# Pivot the DataFrame
pivot_df = grouped_filtered.pivot(index='Inventors', columns='Year Group', values=['Publication Count', 'Total Citations',  'Average Citation Count'])

# Fill NaN values with 0
#pivot_df.fillna(0, inplace=True)
# Save data
pivot_df.to_pickle(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/inventors_list_count_citation_group_year.pkl')
pivot_df.to_csv(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/inventors_list_count_citation_group_year.csv')

# Print the pivoted DataFrame
pivot_df


Publication Count                            \
Year Group                    [2003, 2008) [2008, 2013) [2013, 2018)   
Inventors                                                              
A MINA                                 NaN          NaN          NaN   
A P NETHRAVATHI                        NaN          NaN          NaN   
AAGE LOESET GEIR                       NaN          1.0          NaN   
ABAEV IGOR V                           NaN          NaN          3.0   
ABAEV IGOR VALENTINOVICH               NaN          NaN          1.0   
...                                    ...          ...          ...   
유상열                                    NaN          NaN          NaN   
이상훈                                    NaN          1.0          NaN   
이유나                                    NaN          NaN          NaN   
조도형                                    NaN          NaN          NaN   
차여연                                    NaN          NaN          NaN   

                                                   Total Citations  \
Year Group               [2018, 2023) [2023, 2028)    [2003, 2008)   
Inventors                                                            
A MINA                            1.0          NaN             NaN   
A P NETHRAVATHI                   1.0          NaN             NaN   
AAGE LOESET GEIR                  NaN          NaN             NaN   
ABAEV IGOR V                      NaN          NaN             NaN   
ABAEV IGOR VALENTINOVICH          NaN          NaN             NaN   
...                               ...          ...             ...   
유상열                               1.0          NaN             NaN   
이상훈                               NaN          NaN             NaN   
이유나                               1.0          NaN             NaN   
조도형                               1.0          NaN             NaN   
차여연                               1.0          NaN             NaN   

                                                                              \
Year Group               [2008, 2013) [2013, 2018) [2018, 2023) [2023, 2028)   
Inventors                                                                      
A MINA                            NaN          NaN          4.0          NaN   
A P NETHRAVATHI                   NaN          NaN          0.0          NaN   
AAGE LOESET GEIR                  2.0          NaN          NaN          NaN   
ABAEV IGOR V                      NaN          1.0          NaN          NaN   
ABAEV IGOR VALENTINOVICH          NaN          1.0          NaN          NaN   
...                               ...          ...          ...          ...   
유상열                               NaN          NaN          0.0          NaN   
이상훈                               0.0          NaN          NaN          NaN   
이유나                               NaN          NaN          0.0          NaN   
조도형                               NaN          NaN          0.0          NaN   
차여연                               NaN          NaN          0.0          NaN   

                         Average Citation Count                            \
Year Group                         [2003, 2008) [2008, 2013) [2013, 2018)   
Inventors                                                                   
A MINA                                      NaN          NaN          NaN   
A P NETHRAVATHI                             NaN          NaN          NaN   
AAGE LOESET GEIR                            NaN          2.0          NaN   
ABAEV IGOR V                                NaN          NaN     0.333333   
ABAEV IGOR VALENTINOVICH                    NaN          NaN     1.000000   
...                                         ...          ...          ...   
유상열                                         NaN          NaN          NaN   
이상훈                                         NaN          0.0          NaN   
이유나                                         NaN          NaN       

# Jurisdiction

In [18]:
#Import jurisdiction_data
jurisdiction_data = pd.read_pickle(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/topic_probability_patent.pkl').reset_index()[
    ['Lens ID','Publication Year','Jurisdiction','Cited by Patent Count']]
# Convert 'Publication Year' to integers
jurisdiction_data['Publication Year'] = jurisdiction_data['Publication Year'].astype(int)

jurisdiction_data = jurisdiction_data.set_index(['Lens ID','Publication Year','Cited by Patent Count'])
jurisdiction_data

,,,Jurisdiction
Lens ID,Publication Year,Cited by Patent Count,
088-886-194-310-021,2006,7,FR
147-331-231-570-922,2007,2,WO
148-164-161-664-368,2007,0,WO
161-684-752-020-329,2004,1,JP
089-102-410-621-069,2004,0,JP
...,...,...,...
179-151-089-030-564,2019,0,CZ
014-594-233-943-253,2020,1,CZ
115-179-292-453-581,2006,1,AU


In [19]:
import pandas as pd

# Your DataFrame is named jurisdiction_data
jurisdiction_list = jurisdiction_data['Jurisdiction'].str.split(';;').explode().str.strip()

# Count the unique jurisdiction names
unique_jurisdiction_count = jurisdiction_list.nunique()

print('The number of unique jurisdiction is:', unique_jurisdiction_count)

#
jurisdiction_list2 = jurisdiction_list.to_frame().reset_index()

# and sum citations within each group
grouped = jurisdiction_list2.groupby(['Jurisdiction']).agg({'Lens ID': 'count', 'Cited by Patent Count': 'sum'}).reset_index()

grouped['%'] = grouped['Lens ID']/jurisdiction_data['Jurisdiction'].count()*100
grouped['Average Citation Count'] = grouped['Cited by Patent Count']/grouped['Lens ID']
# Rename columns
grouped.columns = ['Jurisdiction', 'count','citation', '%',  'Average Citation Count']

# Save data
grouped.to_pickle(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/jurisdiction_count_citation.pkl')
grouped.to_csv(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/jurisdiction_count_citation.csv')

grouped#.head(50)

The number of unique jurisdiction is: 37


,Jurisdiction,count,citation,%,Average Citation Count
0,AP,1,0,0.019577,0.000000
1,AU,71,4,1.389977,0.056338
2,BG,2,0,0.039154,0.000000
3,CA,66,6,1.292091,0.090909
4,CN,2454,4831,48.042287,1.968623
5,CZ,9,1,0.176194,0.111111
6,DE,6,30,0.117463,5.000000
7,DK,1,0,0.019577,0.000000
8,EA,5,6,0.097886,1.200000
9,EP,63,63,1.233359,1.000000


In [20]:
# Reset the index to make 'Publication Year' a regular column
jurisdiction_list2 = jurisdiction_list.to_frame().reset_index()
# Create a new column, 'Year Group', to indicate which 5-year group each row belongs to
jurisdiction_list2['Year Group'] = pd.cut(jurisdiction_list2['Publication Year'], bins=range(2003, 2035, 5), right=False)


# Group by 'Year Group' and '......', count number of publications
# and sum citations within each group
grouped = jurisdiction_list2.groupby(['Year Group','Jurisdiction']).agg({'Lens ID': 'count', 'Cited by Patent Count': 'sum'}).reset_index()
grouped['Average Citation Count'] = grouped['Cited by Patent Count']/grouped['Lens ID']
# Create a boolean mask to identify rows with 0 in the 'Lens ID' column
mask = (grouped['Lens ID'] == 0)

# Use the mask to drop rows with 0 in the 'Lens ID' column
grouped_filtered = grouped[~mask]

# Rename columns
grouped_filtered.columns = ['Year Group','Jurisdiction', 'Publication Count', 'Total Citations',  'Average Citation Count']

grouped_filtered


,Year Group,Jurisdiction,Publication Count,Total Citations,Average Citation Count
0,"[2003, 2008)",AP,1,0,0.000000
1,"[2003, 2008)",AU,6,1,0.166667
2,"[2003, 2008)",BG,1,0,0.000000
3,"[2003, 2008)",CA,12,1,0.083333
4,"[2003, 2008)",CN,107,331,3.093458
...,...,...,...,...,...
166,"[2023, 2028)",KR,6,0,0.000000
178,"[2023, 2028)",SK,1,0,0.000000
180,"[2023, 2028)",TW,2,0,0.000000
182,"[2023, 2028)",US,29,0,0.000000


In [21]:
# Pivot the DataFrame
pivot_df = grouped_filtered.pivot(index='Jurisdiction', columns='Year Group', values=['Publication Count', 'Total Citations',  'Average Citation Count'])

# Fill NaN values with 0
#pivot_df.fillna(0, inplace=True)
# Save data
pivot_df.to_pickle(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/jurisdiction_list_count_citation_group_year.pkl')
pivot_df.to_csv(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/jurisdiction_list_count_citation_group_year.csv')

# Print the pivoted DataFrame
pivot_df


Publication Count                                         \
Year Group        [2003, 2008) [2008, 2013) [2013, 2018) [2018, 2023)   
Jurisdiction                                                            
AP                         1.0          NaN          NaN          NaN   
AU                         6.0          9.0         20.0         33.0   
BG                         1.0          1.0          NaN          NaN   
CA                        12.0         11.0         13.0         30.0   
CN                       107.0        278.0        679.0       1318.0   
CZ                         1.0          2.0          1.0          4.0   
DE                         4.0          2.0          NaN          NaN   
DK                         1.0          NaN          NaN          NaN   
EA                         3.0          1.0          NaN          1.0   
EP                        17.0         13.0          7.0         20.0   
ES                         1.0          3.0          1.0          NaN   
FR                         1.0          NaN          NaN          NaN   
GB                        13.0         10.0          8.0          2.0   
GE                         9.0          8.0          1.0          8.0   
GR                         NaN          1.0          NaN          NaN   
HU                         NaN          NaN          1.0          NaN   
IN                         1.0          NaN          3.0          NaN   
JP                        51.0         46.0         39.0         30.0   
KR                        38.0         69.0        102.0        145.0   
LV                         NaN          2.0          3.0          NaN   
MX                         6.0          5.0          8.0         11.0   
MY                         NaN          2.0          3.0          1.0   
NL                         NaN          NaN          NaN          2.0   
NZ                        17.0         11.0          2.0          1.0   
PH                         NaN          NaN          4.0          NaN   
PL                         NaN          3.0          7.0         17.0   
RO                         NaN          NaN          NaN          2.0   
RU                        53.0         55.0         52.0        132.0   
SG                         NaN          2.0          2.0          1.0   
SI                         NaN          NaN          2.0          NaN   
SK                         NaN          NaN          NaN          NaN   
TN                         1.0          2.0          2.0          NaN   
TW                         6.0          3.0          6.0          7.0   
UA                         9.0         16.0          9.0          NaN   
US                       139.0        142.0        158.0        235.0   
WO                       130.0        113.0        103.0        260.0   
ZA                         NaN          NaN          NaN          5.0   

                          Total Citations                            \
Year Group   [2023, 2028)    [2003, 2008) [2008, 2013) [2013, 2018)   
Jurisdiction                                                          
AP                    NaN             0.0          NaN          NaN   
AU                    3.0             1.0          0.0          2.0   
BG                    NaN             0.0          0.0          NaN   
CA                    NaN             1.0          1.0          2.0   
CN                   72.0           331.0        974.0       2038.0   
CZ                    1.0             0.0          0.0          0.0   
DE                    NaN            24.0          6.0          NaN   
DK                    NaN             0.0          NaN          NaN   
EA                    NaN             6.0          0.0          NaN   
EP                    6.0            29.0         29.0          2.0   
ES                    NaN             0.0          0.0          0.0   
FR                    NaN             7.0          NaN          NaN   
GB                    5.0  

# Kind

In [22]:
#Import kind_data
kind_data = pd.read_pickle(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/topic_probability_patent.pkl').reset_index()[
    ['Lens ID','Publication Year','Kind','Cited by Patent Count']]
# Convert 'Publication Year' to integers
kind_data['Publication Year'] = kind_data['Publication Year'].astype(int)

kind_data = kind_data.set_index(['Lens ID','Publication Year','Cited by Patent Count'])
kind_data

,,,Kind
Lens ID,Publication Year,Cited by Patent Count,
088-886-194-310-021,2006,7,A1
147-331-231-570-922,2007,2,A1
148-164-161-664-368,2007,0,A1
161-684-752-020-329,2004,1,A
089-102-410-621-069,2004,0,A
...,...,...,...
179-151-089-030-564,2019,0,U1
014-594-233-943-253,2020,1,U1
115-179-292-453-581,2006,1,A1


In [23]:
import pandas as pd

# Your DataFrame is named kind_data
kind_list = kind_data['Kind'].str.split(';;').explode().str.strip()

# Count the unique kind names
unique_kind_count = kind_list.nunique()

print('The number of unique kind is:', unique_kind_count)

#
kind_list2 = kind_list.to_frame().reset_index()

# and sum citations within each group
grouped = kind_list2.groupby(['Kind']).agg({'Lens ID': 'count', 'Cited by Patent Count': 'sum'}).reset_index()

grouped['%'] = grouped['Lens ID']/kind_data['Kind'].count()*100
grouped['Average Citation Count'] = grouped['Cited by Patent Count']/grouped['Lens ID']
# Rename columns
grouped.columns = ['Kind', 'count','citation', '%',  'Average Citation Count']

# Save data
grouped.to_pickle(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/kind_count_citation.pkl')
grouped.to_csv(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/kind_count_citation.csv')

grouped#.head(50)

The number of unique kind is: 20


,Kind,count,citation,%,Average Citation Count
0,A,2634,5296,51.566171,2.010630
1,A0,2,1,0.039154,0.500000
2,A1,1061,4689,20.771339,4.419416
3,A2,102,437,1.996868,4.284314
4,A3,106,2,2.075176,0.018868
5,A4,12,1,0.234926,0.083333
6,A8,10,0,0.195771,0.000000
7,A9,10,31,0.195771,3.100000
8,B,59,8,1.155051,0.135593
9,B1,120,428,2.349256,3.566667


In [24]:
# Reset the index to make 'Publication Year' a regular column
kind_list2 = kind_list.to_frame().reset_index()
# Create a new column, 'Year Group', to indicate which 5-year group each row belongs to
kind_list2['Year Group'] = pd.cut(kind_list2['Publication Year'], bins=range(2003, 2035, 5), right=False)


# Group by 'Year Group' and '......', count number of publications
# and sum citations within each group
grouped = kind_list2.groupby(['Year Group','Kind']).agg({'Lens ID': 'count', 'Cited by Patent Count': 'sum'}).reset_index()
grouped['Average Citation Count'] = grouped['Cited by Patent Count']/grouped['Lens ID']
# Create a boolean mask to identify rows with 0 in the 'Lens ID' column
mask = (grouped['Lens ID'] == 0)

# Use the mask to drop rows with 0 in the 'Lens ID' column
grouped_filtered = grouped[~mask]

# Rename columns
grouped_filtered.columns = ['Year Group','Kind', 'Publication Count', 'Total Citations',  'Average Citation Count']

grouped_filtered


,Year Group,Kind,Publication Count,Total Citations,Average Citation Count
0,"[2003, 2008)",A,221,525,2.375566
1,"[2003, 2008)",A0,1,0,0.000000
2,"[2003, 2008)",A1,170,2310,13.588235
3,"[2003, 2008)",A2,52,283,5.442308
4,"[2003, 2008)",A3,39,0,0.000000
...,...,...,...,...,...
89,"[2023, 2028)",B1,1,0,0.000000
90,"[2023, 2028)",B2,12,0,0.000000
91,"[2023, 2028)",B6,2,0,0.000000
96,"[2023, 2028)",D0,2,0,0.000000


In [25]:
# Pivot the DataFrame
pivot_df = grouped_filtered.pivot(index='Kind', columns='Year Group', values=['Publication Count', 'Total Citations',  'Average Citation Count'])

# Fill NaN values with 0
#pivot_df.fillna(0, inplace=True)
# Save data
pivot_df.to_pickle(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/kind_list_count_citation_group_year.pkl')
pivot_df.to_csv(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/kind_list_count_citation_group_year.csv')

# Print the pivoted DataFrame
pivot_df


Publication Count                                         \
Year Group      [2003, 2008) [2008, 2013) [2013, 2018) [2018, 2023)   
Kind                                                                  
A                      221.0        372.0        687.0       1286.0   
A0                       1.0          NaN          NaN          1.0   
A1                     170.0        178.0        214.0        436.0   
A2                      52.0         25.0          9.0         14.0   
A3                      39.0         36.0         15.0         13.0   
A4                       1.0          1.0          NaN          9.0   
A8                       NaN          2.0          NaN          8.0   
A9                       1.0          2.0          2.0          4.0   
B                       10.0         41.0          1.0          7.0   
B1                      27.0         11.0         21.0         60.0   
B2                      27.0         48.0         68.0         77.0   
B6                       1.0          NaN          NaN          1.0   
C                        2.0          5.0          4.0         10.0   
C1                      20.0         38.0         33.0        114.0   
C2                      35.0         19.0         20.0         17.0   
C9                       NaN          NaN          NaN          1.0   
D0                       3.0          4.0          3.0          1.0   
U                        5.0         16.0        157.0        196.0   
U1                       NaN          3.0          1.0          3.0   
Y                       13.0          9.0          1.0          7.0   

                        Total Citations                            \
Year Group [2023, 2028)    [2003, 2008) [2008, 2013) [2013, 2018)   
Kind                                                                
A                  68.0           525.0       1289.0       1996.0   
A0                  NaN             0.0          NaN          NaN   
A1                 63.0          2310.0        827.0       1017.0   
A2                  2.0           283.0        128.0         22.0   
A3                  3.0             0.0          2.0          0.0   
A4                  1.0             0.0          0.0          NaN   
A8                  NaN             NaN          0.0          NaN   
A9                  1.0             0.0          2.0         29.0   
B                   NaN             1.0          7.0          0.0   
B1                  1.0           234.0         62.0         87.0   
B2                 12.0           393.0        404.0        101.0   
B6                  2.0             0.0          NaN          NaN   
C                   NaN             0.0          3.0          2.0   
C1                  NaN            18.0         33.0         26.0   
C2                  NaN            20.0         17.0         19.0   
C9                  NaN             NaN          NaN          NaN   
D0                  2.0             0.0          0.0          0.0   
U                  14.0             0.0          2.0        243.0   
U1                  NaN             NaN          0.0          0.0   
Y                   NaN            20.0          1.0          0.0   

                                     Average Citation Count               \
Year Group [2018, 2023) [2023, 2028)           [2003, 2008) [2008, 2013)   
Kind                                                                       
A                1483.0          3.0               2.375566     3.465054   
A0                  1.0          NaN               0.000000          NaN   
A1                535.0          0.0              13.588235     4.646067   
A2                  4.0          0.0               5.442308     5.120000   
A3                  0.0          0.0               0.000000     0.055556   
A4                  1.0          0.0               0.000000     0.000000   
A8                  0.0          NaN                    NaN     0.000000   
A9                  0.0      

# IPCR Classifications

In [26]:
#Import IPCR_classifications_data
IPCR_classifications_data = pd.read_pickle(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/topic_probability_patent.pkl').reset_index()[
    ['Lens ID','Publication Year','IPCR Classifications','Cited by Patent Count']]
# Convert 'Publication Year' to integers
IPCR_classifications_data['Publication Year'] = IPCR_classifications_data['Publication Year'].astype(int)

IPCR_classifications_data = IPCR_classifications_data.set_index(['Lens ID','Publication Year','Cited by Patent Count'])
IPCR_classifications_data

,,,IPCR Classifications
Lens ID,Publication Year,Cited by Patent Count,
088-886-194-310-021,2006,7,A61K38/17;;A61K39/395;;A61K48/00;;A61P9/00;;A6...
147-331-231-570-922,2007,2,G01N33/53;;C07D229/02;;G01N33/15;;G01N33/543;;...
148-164-161-664-368,2007,0,C12N15/09;;C07K14/005;;C12N1/21;;C12N7/00;;C12...
161-684-752-020-329,2004,1,C12N15/09;;C12Q1/04;;C12Q1/34
089-102-410-621-069,2004,0,C12N15/09;;A01N25/28;;A01N63/00;;C12N1/20;;C12...
...,...,...,...
179-151-089-030-564,2019,0,C12Q1/70;;C12N7/02
014-594-233-943-253,2020,1,C12Q1/70;;C12R1/44
115-179-292-453-581,2006,1,A61K38/00


In [27]:
import pandas as pd

# Your DataFrame is named IPCR_classifications_data
IPCR_classifications_list = IPCR_classifications_data['IPCR Classifications'].str.split(';;').explode().str.strip()

# Count the unique IPCR_classifications names
unique_IPCR_classifications_count = IPCR_classifications_list.nunique()

print('The number of unique IPCR_classifications is:', unique_IPCR_classifications_count)

#
IPCR_classifications_list2 = IPCR_classifications_list.to_frame().reset_index()

# and sum citations within each group
grouped = IPCR_classifications_list2.groupby(['IPCR Classifications']).agg({'Lens ID': 'count', 'Cited by Patent Count': 'sum'}).reset_index()

grouped['%'] = grouped['Lens ID']/IPCR_classifications_data['IPCR Classifications'].count()*100
grouped['Average Citation Count'] = grouped['Cited by Patent Count']/grouped['Lens ID']
# Rename columns
grouped.columns = ['IPCR Classifications', 'count','citation', '%',  'Average Citation Count']

# Save data
grouped.to_pickle(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/IPCR_classifications_count_citation.pkl')
grouped.to_csv(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/IPCR_classifications_count_citation.csv')

grouped##.head(50)

The number of unique IPCR_classifications is: 2401


,IPCR Classifications,count,citation,%,Average Citation Count
0,A01B79/02,1,1,0.019743,1.000000
1,A01C1/00,1,0,0.019743,0.000000
2,A01G1/04,1,0,0.019743,0.000000
3,A01G13/00,6,13,0.118460,2.166667
4,A01G13/10,1,0,0.019743,0.000000
...,...,...,...,...,...
2396,H05B3/00,1,0,0.019743,0.000000
2397,H05B45/00,1,0,0.019743,0.000000
2398,H05B6/00,1,50,0.019743,50.000000
2399,H05K1/02,1,1,0.019743,1.000000


In [28]:
# Reset the index to make 'Publication Year' a regular column
IPCR_classifications_list2 = IPCR_classifications_list.to_frame().reset_index()
# Create a new column, 'Year Group', to indicate which 5-year group each row belongs to
IPCR_classifications_list2['Year Group'] = pd.cut(IPCR_classifications_list2['Publication Year'], bins=range(2003, 2035, 5), right=False)


# Group by 'Year Group' and '......', count number of publications
# and sum citations within each group
grouped = IPCR_classifications_list2.groupby(['Year Group','IPCR Classifications']).agg({'Lens ID': 'count', 'Cited by Patent Count': 'sum'}).reset_index()
grouped['Average Citation Count'] = grouped['Cited by Patent Count']/grouped['Lens ID']
# Create a boolean mask to identify rows with 0 in the 'Lens ID' column
mask = (grouped['Lens ID'] == 0)

# Use the mask to drop rows with 0 in the 'Lens ID' column
grouped_filtered = grouped[~mask]

# Rename columns
grouped_filtered.columns = ['Year Group','IPCR Classifications', 'Publication Count', 'Total Citations',  'Average Citation Count']

grouped_filtered


,Year Group,IPCR Classifications,Publication Count,Total Citations,Average Citation Count
13,"[2003, 2008)",A01G31/00,2,11,5.5
15,"[2003, 2008)",A01G7/00,1,7,7.0
21,"[2003, 2008)",A01H1/00,2,2,1.0
22,"[2003, 2008)",A01H1/02,1,0,0.0
25,"[2003, 2008)",A01H4/00,2,2,1.0
...,...,...,...,...,...
11931,"[2023, 2028)",G16B40/20,1,0,0.0
11938,"[2023, 2028)",G16H20/10,2,0,0.0
11940,"[2023, 2028)",G16H40/20,1,0,0.0
11941,"[2023, 2028)",G16H40/63,1,0,0.0


In [29]:
# Pivot the DataFrame
pivot_df = grouped_filtered.pivot(index='IPCR Classifications', columns='Year Group', values=['Publication Count', 'Total Citations',  'Average Citation Count'])

# Fill NaN values with 0
#pivot_df.fillna(0, inplace=True)
# Save data
pivot_df.to_pickle(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/IPCR_classifications_list_count_citation_group_year.pkl')
pivot_df.to_csv(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/IPCR_classifications_list_count_citation_group_year.csv')

# Print the pivoted DataFrame
pivot_df


Publication Count                                         \
Year Group                [2003, 2008) [2008, 2013) [2013, 2018) [2018, 2023)   
IPCR Classifications                                                            
A01B79/02                          NaN          NaN          1.0          NaN   
A01C1/00                           NaN          NaN          NaN          1.0   
A01G1/04                           NaN          NaN          1.0          NaN   
A01G13/00                          NaN          1.0          NaN          3.0   
A01G13/10                          NaN          NaN          NaN          1.0   
...                                ...          ...          ...          ...   
H05B3/00                           NaN          NaN          NaN          1.0   
H05B45/00                          NaN          NaN          NaN          1.0   
H05B6/00                           1.0          NaN          NaN          NaN   
H05K1/02                           NaN          NaN          NaN          1.0   
H05K7/20                           NaN          NaN          NaN          1.0   

                                  Total Citations                            \
Year Group           [2023, 2028)    [2003, 2008) [2008, 2013) [2013, 2018)   
IPCR Classifications                                                          
A01B79/02                     NaN             NaN          NaN          1.0   
A01C1/00                      NaN             NaN          NaN          NaN   
A01G1/04                      NaN             NaN          NaN          0.0   
A01G13/00                     2.0             NaN          9.0          NaN   
A01G13/10                     NaN             NaN          NaN          NaN   
...                           ...             ...          ...          ...   
H05B3/00                      NaN             NaN          NaN          NaN   
H05B45/00                     NaN             NaN          NaN          NaN   
H05B6/00                      NaN            50.0          NaN          NaN   
H05K1/02                      NaN             NaN          NaN          NaN   
H05K7/20                      NaN             NaN          NaN          NaN   

                                               Average Citation Count  \
Year Group           [2018, 2023) [2023, 2028)           [2003, 2008)   
IPCR Classifications                                                    
A01B79/02                     NaN          NaN                    NaN   
A01C1/00                      0.0          NaN                    NaN   
A01G1/04                      NaN          NaN                    NaN   
A01G13/00                     4.0          0.0                    NaN   
A01G13/10                     0.0          NaN                    NaN   
...                           ...          ...                    ...   
H05B3/00                      0.0          NaN                    NaN   
H05B45/00                     0.0          NaN                    NaN   
H05B6/00                      NaN          NaN                   50.0   
H05K1/02                      1.0          NaN                    NaN   
H05K7/20                      1.0          NaN                    NaN   

                                                                          
Year Group           [2008, 2013) [2013, 2018) [2018, 2023) [2023, 2028)  
IPCR Classifications                                                      
A01B79/02                     NaN          1.0          NaN          NaN  
A01C1/00                      NaN          NaN     0.000000          NaN  
A01G1/04                      NaN          0.0          NaN          NaN  
A01G13/00                     9.0          NaN     1.333333          0.0  
A01G13/10                     NaN          NaN     0.000000          NaN  
...                           ...          ...          ...          ...  
H05B3/00                      NaN          NaN     0.000000          NaN  
H05B45

# Legal Status

In [30]:
#Import legal_status_data
legal_status_data = pd.read_pickle(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/topic_probability_patent.pkl').reset_index()[
    ['Lens ID','Publication Year','Legal Status','Cited by Patent Count']]
# Convert 'Publication Year' to integers
legal_status_data['Publication Year'] = legal_status_data['Publication Year'].astype(int)

legal_status_data = legal_status_data.set_index(['Lens ID','Publication Year','Cited by Patent Count'])
legal_status_data

,,,Legal Status
Lens ID,Publication Year,Cited by Patent Count,
088-886-194-310-021,2006,7,DISCONTINUED
147-331-231-570-922,2007,2,PENDING
148-164-161-664-368,2007,0,PENDING
161-684-752-020-329,2004,1,EXPIRED
089-102-410-621-069,2004,0,EXPIRED
...,...,...,...
179-151-089-030-564,2019,0,ACTIVE
014-594-233-943-253,2020,1,ACTIVE
115-179-292-453-581,2006,1,DISCONTINUED


In [31]:
import pandas as pd

# Your DataFrame is named legal_status_data
legal_status_list = legal_status_data['Legal Status'].str.split(';;').explode().str.strip()

# Count the unique legal_status names
unique_legal_status_count = legal_status_list.nunique()

print('The number of unique legal_status is:', unique_legal_status_count)

#
legal_status_list2 = legal_status_list.to_frame().reset_index()

# and sum citations within each group
grouped = legal_status_list2.groupby(['Legal Status']).agg({'Lens ID': 'count', 'Cited by Patent Count': 'sum'}).reset_index()

grouped['%'] = grouped['Lens ID']/legal_status_data['Legal Status'].count()*100

grouped['Average Citation Count'] = grouped['Cited by Patent Count']/grouped['Lens ID']

# Rename columns
grouped.columns = ['Legal Status', 'count','citation', '%',  'Average Citation Count']

# Save data
grouped.to_pickle(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/legal_status_count_citation.pkl')
grouped.to_csv(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/legal_status_count_citation.csv')

grouped#.head(50)

The number of unique legal_status is: 7


,Legal Status,count,citation,%,Average Citation Count
0,ACTIVE,1836,3944,35.943618,2.148148
1,DISCONTINUED,925,3784,18.108849,4.090811
2,EXPIRED,213,865,4.169930,4.061033
3,INACTIVE,644,1586,12.607674,2.462733
4,PATENTED,6,8,0.117463,1.333333
5,PENDING,1479,2165,28.954581,1.463827
6,UNKNOWN,5,0,0.097886,0.000000


In [32]:
# Reset the index to make 'Publication Year' a regular column
legal_status_list2 = legal_status_list.to_frame().reset_index()
# Create a new column, 'Year Group', to indicate which 5-year group each row belongs to
legal_status_list2['Year Group'] = pd.cut(legal_status_list2['Publication Year'], bins=range(2003, 2035, 5), right=False)


# Group by 'Year Group' and '......', count number of publications
# and sum citations within each group
grouped = legal_status_list2.groupby(['Year Group','Legal Status']).agg({'Lens ID': 'count', 'Cited by Patent Count': 'sum'}).reset_index()

grouped['Average Citation Count'] = grouped['Cited by Patent Count']/grouped['Lens ID']

# Create a boolean mask to identify rows with 0 in the 'Lens ID' column
mask = (grouped['Lens ID'] == 0)

# Use the mask to drop rows with 0 in the 'Lens ID' column
grouped_filtered = grouped[~mask]

# Rename columns
grouped_filtered.columns = ['Year Group','Legal Status', 'Publication Count', 'Total Citations',  'Average Citation Count']

grouped_filtered


,Year Group,Legal Status,Publication Count,Total Citations,Average Citation Count
0,"[2003, 2008)",ACTIVE,38,280,7.368421
1,"[2003, 2008)",DISCONTINUED,162,1636,10.098765
2,"[2003, 2008)",EXPIRED,156,797,5.108974
3,"[2003, 2008)",INACTIVE,97,414,4.268041
4,"[2003, 2008)",PATENTED,6,8,1.333333
5,"[2003, 2008)",PENDING,167,669,4.005988
6,"[2003, 2008)",UNKNOWN,2,0,0.000000
7,"[2008, 2013)",ACTIVE,191,865,4.528796
8,"[2008, 2013)",DISCONTINUED,217,819,3.774194
9,"[2008, 2013)",EXPIRED,44,63,1.431818


In [33]:
# Pivot the DataFrame
pivot_df = grouped_filtered.pivot(index='Legal Status', columns='Year Group', values=['Publication Count', 'Total Citations',  'Average Citation Count'])

# Fill NaN values with 0
#pivot_df.fillna(0, inplace=True)
# Save data
pivot_df.to_pickle(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/legal_status_list_count_citation_group_year.pkl')
pivot_df.to_csv(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/legal_status_list_count_citation_group_year.csv')

# Print the pivoted DataFrame
pivot_df


Publication Count                                         \
Year Group        [2003, 2008) [2008, 2013) [2013, 2018) [2018, 2023)   
Legal Status                                                            
ACTIVE                    38.0        191.0        509.0       1068.0   
DISCONTINUED             162.0        217.0        332.0        212.0   
EXPIRED                  156.0         44.0         13.0          NaN   
INACTIVE                  97.0        215.0        226.0        106.0   
PATENTED                   6.0          NaN          NaN          NaN   
PENDING                  167.0        142.0        156.0        879.0   
UNKNOWN                    2.0          1.0          NaN          NaN   

                          Total Citations                            \
Year Group   [2023, 2028)    [2003, 2008) [2008, 2013) [2013, 2018)   
Legal Status                                                          
ACTIVE               30.0           280.0        865.0       1572.0   
DISCONTINUED          2.0          1636.0        819.0       1041.0   
EXPIRED               NaN           797.0         63.0          5.0   
INACTIVE              NaN           414.0        669.0        420.0   
PATENTED              NaN             8.0          NaN          NaN   
PENDING             135.0           669.0        361.0        504.0   
UNKNOWN               2.0             0.0          0.0          NaN   

                                       Average Citation Count               \
Year Group   [2018, 2023) [2023, 2028)           [2003, 2008) [2008, 2013)   
Legal Status                                                                 
ACTIVE             1227.0          0.0               7.368421     4.528796   
DISCONTINUED        288.0          0.0              10.098765     3.774194   
EXPIRED               NaN          NaN               5.108974     1.431818   
INACTIVE             83.0          NaN               4.268041     3.111628   
PATENTED              NaN          NaN               1.333333          NaN   
PENDING             628.0          3.0               4.005988     2.542254   
UNKNOWN               NaN          0.0               0.000000     0.000000   

                                                     
Year Group   [2013, 2018) [2018, 2023) [2023, 2028)  
Legal Status                                         
ACTIVE           3.088409     1.148876     0.000000  
DISCONTINUED     3.135542     1.358491     0.000000  
EXPIRED          0.384615          NaN          NaN  
INACTIVE         1.858407     0.783019          NaN  
PATENTED              NaN          NaN          NaN  
PENDING          3.230769     0.714448     0.022222  
UNKNOWN               NaN          NaN     0.000000

# topics

In [34]:
topics_data_citation = pd.read_pickle(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/topic_probability_patent.pkl').reset_index()[
    ['Lens ID','Publication Year','Topic N°','Cited by Patent Count']]#.reset_index()
topics_data_citation

,Lens ID,Publication Year,Topic N°,Cited by Patent Count
0,088-886-194-310-021,2006,4,7
1,147-331-231-570-922,2007,2,2
2,148-164-161-664-368,2007,2,0
3,161-684-752-020-329,2004,0,1
4,089-102-410-621-069,2004,5,0
...,...,...,...,...
5103,179-151-089-030-564,2019,0,0
5104,014-594-233-943-253,2020,5,1
5105,115-179-292-453-581,2006,5,1
5106,069-774-923-021-970,2005,5,0


In [35]:
#citation_data = data.copy().reset_index().set_index(['Lens ID','Publication Year'])[['Cited by Patent Count']]
#citation_data

In [36]:
#topics_data_citation = pd.concat([topics_data, citation_data], axis=1)
#topics_data_citation

In [37]:
# Count the unique topic names
unique_topic_count = topics_data_citation['Topic N°'].nunique()

print('The number of unique topics is:', unique_topic_count)

#
topics_data_citation2 = topics_data_citation.reset_index()

# and sum citations within each group
grouped = topics_data_citation2.groupby(['Topic N°']).agg({'Lens ID': 'count', 'Cited by Patent Count': 'sum'}).reset_index()

grouped['%'] = grouped['Lens ID']/topics_data_citation['Topic N°'].count()*100

grouped['Average Citation Count'] = grouped['Cited by Patent Count']/grouped['Lens ID']

# Rename columns
grouped.columns = ['Topic N°', 'Publication Count', 'Total Citations', '%','Average Citation Count']

# Save data
grouped.to_pickle(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/topics_counts_citation.pkl')
grouped.to_csv(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/topics_counts_citation.csv')

grouped#.head(50)

The number of unique topics is: 8


,Topic N°,Publication Count,Total Citations,%,Average Citation Count
0,0,266,524,5.207518,1.969925
1,1,391,397,7.654659,1.015345
2,2,1310,4050,25.646045,3.091603
3,3,596,1132,11.667972,1.899329
4,4,260,740,5.090055,2.846154
5,5,1512,3552,29.600626,2.349206
6,6,175,474,3.425998,2.708571
7,7,598,1483,11.707126,2.479933


In [38]:
topics_data2 = topics_data_citation.reset_index()#.set_index('Lens ID')
# Create a new column, 'Year Group', to indicate which 5-year group each row belongs to
topics_data2['Year Group'] = pd.cut(topics_data2['Publication Year'], bins=range(2003, 2035, 5), right=False)

# Group by 'Year Group' and 'Topic N°', count number of publications
# and sum citations within each group
grouped = topics_data2.groupby(['Year Group','Topic N°']).agg({'Lens ID': 'count', 'Cited by Patent Count': 'sum'}).reset_index()

grouped['Average Citation Count'] = grouped['Cited by Patent Count']/grouped['Lens ID']

# Create a boolean mask to identify rows with 0 in the 'Lens ID' column
mask = (grouped['Lens ID'] == 0)

# Use the mask to drop rows with 0 in the 'Lens ID' column
grouped_filtered = grouped[~mask]

# Rename columns
grouped_filtered.columns = ['Year Group','Topic N°', 'Publication Count', 'Total Citations',  'Average Citation Count']

grouped_filtered


,Year Group,Topic N°,Publication Count,Total Citations,Average Citation Count
0,"[2003, 2008)",0,35,229,6.542857
1,"[2003, 2008)",1,9,22,2.444444
2,"[2003, 2008)",2,292,2019,6.914384
3,"[2003, 2008)",3,40,218,5.450000
4,"[2003, 2008)",4,56,292,5.214286
5,"[2003, 2008)",5,116,636,5.482759
6,"[2003, 2008)",6,27,197,7.296296
7,"[2003, 2008)",7,53,191,3.603774
8,"[2008, 2013)",0,27,72,2.666667
9,"[2008, 2013)",1,9,16,1.777778


In [39]:
# Pivot the DataFrame
pivot_df = grouped_filtered.pivot(index='Topic N°', columns='Year Group', values=['Publication Count', 'Total Citations',  'Average Citation Count'])

# Fill NaN values with 0
#pivot_df.fillna(0, inplace=True)
# Save data
pivot_df.to_pickle(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/topics_list_count_citation_group_year.pkl')
pivot_df.to_csv(f'/content/drive/MyDrive/YORKU PROJECT/JUDE/BACTERIOPHAGE/topics_list_count_citation_group_year.csv')

# Print the pivoted DataFrame
pivot_df


Publication Count                                         \
Year Group      [2003, 2008) [2008, 2013) [2013, 2018) [2018, 2023)   
Topic N°                                                              
0                       35.0         27.0         51.0        144.0   
1                        9.0          9.0        150.0        209.0   
2                      292.0        275.0        289.0        427.0   
3                       40.0         88.0        199.0        248.0   
4                       56.0         48.0         51.0         97.0   
5                      116.0        219.0        299.0        816.0   
6                       27.0         42.0         42.0         55.0   
7                       53.0        102.0        155.0        269.0   

                        Total Citations                            \
Year Group [2023, 2028)    [2003, 2008) [2008, 2013) [2013, 2018)   
Topic N°                                                            
0                   9.0           229.0         72.0        124.0   
1                  14.0            22.0         16.0        288.0   
2                  27.0          2019.0        803.0        748.0   
3                  21.0           218.0        200.0        508.0   
4                   8.0           292.0        211.0        156.0   
5                  62.0           636.0        972.0       1084.0   
6                   9.0           197.0        177.0         86.0   
7                  19.0           191.0        326.0        548.0   

                                     Average Citation Count               \
Year Group [2018, 2023) [2023, 2028)           [2003, 2008) [2008, 2013)   
Topic N°                                                                   
0                  99.0          0.0               6.542857     2.666667   
1                  71.0          0.0               2.444444     1.777778   
2                 480.0          0.0               6.914384     2.920000   
3                 206.0          0.0               5.450000     2.272727   
4                  81.0          0.0               5.214286     4.395833   
5                 858.0          2.0               5.482759     4.438356   
6                  14.0          0.0               7.296296     4.214286   
7                 417.0          1.0               3.603774     3.196078   

                                                   
Year Group [2013, 2018) [2018, 2023) [2023, 2028)  
Topic N°                                           
0              2.431373     0.687500     0.000000  
1              1.920000     0.339713     0.000000  
2              2.588235     1.124122     0.000000  
3              2.552764     0.830645     0.000000  
4              3.058824     0.835052     0.000000  
5              3.625418     1.051471     0.032258  
6              2.047619     0.254545     0.000000  
7              3.535484     1.550186     0.052632